In [83]:
#STM32 HCI packets functions

#STM32WB HCI command set
ReadVersion = b'\x01\x01\x10\x00'
ReadInfo = b'\x20\x62\xFD\x00'
RxStart = b'\x01\x33\x20\x03\x00\x02\x00'
RxStop = b'\x01\x1F\x20\x00'
ReadRSSI = b'\x01\x32\xFC\x00'
TxPwr = b'\x01\x0F\xFC\x02\x00\x19'
TxStart = b'\x01\x1E\x20\x03\x00\x25\x00'



#Packet construction
#HCI_PACKET_INDICATOR + OPCODE + Parameters number + parameters

#HCI Packet indicator
HCI_PACKET_INDICATOR = b'\x01'

#HCI Packet event
HCI_PACKET_EVENT = 'b\x04'

#HCI OPCODE
HCI_READ_LOCAL_VERSION_INFORMATION = b'\x01\x10'
HCI_LE_ENHANCED_RECEIVER_TEST = b'\x33\x20'
HCI_LE_ENHANCED_TRANSMITTER_TEST = b'\x34\x20'
HCI_LE_TEST_END = b'\x1F\x20'

#ACI OPCODE
ACI_HAL_SET_TX_POWER_LEVEL = b'\x0F\xFC'
ACI_HAL_TONE_START = b'\x15\xFC'
ACI_HAL_TONE_STOP = b'\x16\xFC'
ACI_HAL_READ_RAW_RSSI = b'\x32\xFC'

In [13]:
#Power dictionnary
powerDict = {
    -40: 0x00,
    -20.85: 0x01,
    -19.75: 0x02,
    -18.85: 0x03,
    -17.6: 0x04,
    -16.5: 0x05,
    -15.2: 0x06,
    -14.1: 0x07,
    -13.15: 0x08,
    -12.05: 0x09,
    -10.9: 0x0A,
    -9.9: 0x0B,
    -8.85: 0x0C,
    -7.8: 0x0D,
    -6.9: 0x0E,
    -5.9: 0x0F,
    -4.95: 0x10,
    -4: 0x11,
    -3.15: 0x12,
    -2.45: 0x13,
    -1.8: 0x14,
    -1.3: 0x15,
    -0.85: 0x16,
    -0.5: 0x17,
    -0.15: 0x18,
    0: 0x19,
    1: 0x1A,
    2: 0x1B,
    3: 0x1C,
    4: 0x1D,
    5: 0x1E,
    6: 0x1F
}

In [86]:
#Send packets mesg on serialPort
#Return answer
def SendReceivePackets(serialPort, mesg):
    serialPort.write(mesg)
    header = serialPort.read(2).hex()
    #print("Sending : " + mesg.hex())
    length = serialPort.read(1).hex()
    #print(length)
    dataLength = int(length, base=16)
    #print("Answer length:" + str(dataLength))
    resp = serialPort.read(dataLength).hex()
    #print(resp)
    return header + length + resp

#Extract ACI_HAL_READ_RAW_RSSI rssi in dBm
def ExtractRSSI(resp):
    rssi = int(resp[16:18], base=16)*256 + int(resp[14:16], base=16)
    agc = int(resp[18:20], base=16)
    if(rssi == 0):
        rssi_dbm = 127
    else:
        rssi_dbm = ((agc*6)-127)
        while(rssi > 30):
            rssi_dbm += 6
            rssi = (rssi >> 1)

    rssi_dbm += ((417*rssi + 18080)>>10)
    
    return rssi_dbm

def GetReceivedPackets(resp):
    receivedPackets = int(resp[10:12], base=16)*256 + int(resp[8:10], base=16)
    return receivedPackets

def GetPER(DesensePackets, receivedPackets):
    return float(str("{:.2f}".format(100 - 100*receivedPackets/desensePackets)))

In [87]:
#Generate Tone at channel
#@channel : [0;39]
def StartTone(serialPort, channel):
    size = 2
    if(channel > 39 or channel < 0):
        return 
    mesg = HCI_PACKET_INDICATOR + ACI_HAL_TONE_START + size.to_bytes(1, byteorder='big') + channel.to_bytes(1, byteorder='big') + bytes([0])
    SendReceivePackets(serialPort, mesg)

#Stop Tone
def StopTone(serialPort):
    size = 0
    mesg = HCI_PACKET_INDICATOR + ACI_HAL_TONE_STOP + size.to_bytes(1, byteorder='big')
    SendReceivePackets(serialPort, mesg)
    
#@power is a key from powerDict
#Please refer to powerDict dictionnary for @power parameter
def SetTxPower(serialPort, power):
    size = 2
    if(power in powerDict.keys()):
        mesg = HCI_PACKET_INDICATOR + ACI_HAL_SET_TX_POWER_LEVEL + size.to_bytes(1, byteorder='big') + bytes([0]) + powerDict[power].to_bytes(1, byteorder='big')
        SendReceivePackets(serialPort, mesg)
        
        
#@datalength : payload length in each packet
#@payload : payload type. 0x00 : PRBS9, 0x01 '11110000', 0x02 '1010101010', 0x03 PRBS15, 0x04 all '1', 0x05 all '0'
#0x06 '00001111', 0x07 '01010101'
#@PHY : 0x01 LE 1M, 0x02 LE 2M
def StartTx(serialPort, channel, dataLength, payload, PHY):
    size = 4
    if(channel > 39 or channel < 0):
        return
    if(dataLength < 0x00 or dataLength > 0xFF):
        return
    if(payload < 0x00 or payload > 0x07):
        return
    if(PHY < 0x00 or PHY > 0x02):
        return
    mesg = HCI_PACKET_INDICATOR + HCI_LE_ENHANCED_TRANSMITTER_TEST + size.to_bytes(1, byteorder='big') + channel.to_bytes(1, byteorder='big') + dataLength.to_bytes(1, byteorder='big') + payload.to_bytes(1, byteorder='big') + PHY.to_bytes(1, byteorder='big')
    SendReceivePackets(serialPort, mesg)
    
#Stop Tx
def StopTx(serialPort):
    size = 0
    mesg = HCI_PACKET_INDICATOR + HCI_LE_TEST_END + size.to_bytes(1, byteorder='big')
    SendReceivePackets(serialPort, mesg)
    
#Stop Rx
#Return Received packet number
def StopRx(serialPort):
    size = 0
    mesg = HCI_PACKET_INDICATOR + HCI_LE_TEST_END + size.to_bytes(1, byteorder='big')
    answer = SendReceivePackets(serialPort, mesg)
    receivedPackets = int(answer[16:18], base=16)*256 + int(answer[14:16], base=16)
    return receivedPackets
    
#Start Rx
#@channel : [0;39]
def StartRx(serialPort, channel, PHY):
    size = 3
    if(channel > 39 or channel < 0):
        return
    if(PHY < 0x00 or PHY > 0x02):
        return
    mesg = HCI_PACKET_INDICATOR + HCI_LE_ENHANCED_RECEIVER_TEST + size.to_bytes(1, byteorder='big') + channel.to_bytes(1, byteorder='big') + PHY.to_bytes(1, byteorder='big') + bytes([0])
    SendReceivePackets(serialPort, mesg)
    
#Get RSSI from device
#return RSSI in dBm
def GetRSSI(serialPort):
    size = 0
    mesg = HCI_PACKET_INDICATOR + ACI_HAL_READ_RAW_RSSI + size.to_bytes(1, byteorder='big')
    answer = SendReceivePackets(serialPort, mesg)
    return ExtractRSSI(answer)
    

In [89]:
###############################
##Specify device COMPort#######
STM32ComPort = "COM8"
###############################
import serial
import time
serialPort = serial.Serial(port = STM32ComPort, baudrate = 115200, timeout = 3)


StartRx(serialPort, 0, 0x01)
time.sleep(1)
print(GetRSSI(serialPort))

-108


In [88]:
serialPort.close()